# IA prediction d'humeur BERT

instalation des transdormers

In [1]:
!pip install torch                
!pip install transformers 
!pip install pandas 
!pip install scikit-learn



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import json
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report





SyntaxError: invalid syntax (3736685046.py, line 10)

# 1. Charger les données


In [3]:
import re
import pandas as pd

def load_yelp_dataset(path):
    texts = []
    stars = []

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            star_match = re.search(r"stars:(\d)", line)
            text_match = re.search(r'text:""(.*?)""', line)

            if star_match and text_match:
                texts.append(text_match.group(1))
                stars.append(int(star_match.group(1)))

    return pd.DataFrame({"texte": texts, "stars": stars})


In [7]:
df = load_yelp_dataset("../data/yelp_academic_reviews4students.jsonl")
print(len(df))
print(df.head())


0
Empty DataFrame
Columns: [texte, stars]
Index: []


In [5]:
df = df.sample(200000, random_state=42)


In [6]:
def stars_to_label(stars):
    if stars <= 2:
        return 0   # negatif
    elif stars == 3:
        return 1   # neutre
    else:
        return 2   # positif

df["label"] = df["stars"].apply(stars_to_label)


# 2. Train / Test split


In [7]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["texte"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

# 3. Tokenizer


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3
)

def tokenize(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

Loading weights: 100%|██████████| 199/199 [00:01<00:00, 119.03it/s, Materializing param=bert.pooler.dense.weight]                               
BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those pa


# 4. Dataset PyTorch


In [9]:
class MoodDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = MoodDataset(train_encodings, train_labels)
test_dataset = MoodDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# 5. Modèle


In [10]:
model = CamembertForSequenceClassification.from_pretrained(
    "camembert-base",
    num_labels=3
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)


Loading weights: 100%|██████████| 197/197 [00:01<00:00, 148.77it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
CamembertForSequenceClassification LOAD REPORT from: camembert-base
Key                         | Status     | 
----------------------------+------------+-
roberta.pooler.dense.weight | UNEXPECTED | 
lm_head.layer_norm.bias     | UNEXPECTED | 
lm_head.dense.bias          | UNEXPECTED | 
lm_head.layer_norm.weight   | UNEXPECTED | 
roberta.pooler.dense.bias   | UNEXPECTED | 
lm_head.bias                | UNEXPECTED | 
lm_head.dense.weight        | UNEXPECTED | 
classifier.dense.weight     | MISSING    | 
classifier.out_proj.bias    | MISSING    | 
classifier.dense.bias       | MISSING    | 
classifier.out_proj.weight  | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Conside

# 6. Entraînement


In [11]:
epochs = 3

model.train()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Loss moyenne :", total_loss / len(train_loader))


Epoch 1/3


KeyboardInterrupt: 


# 7. Évaluation


In [ ]:

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("\nRésultats :")
print(classification_report(
    true_labels,
    predictions,
    target_names=["negatif", "neutre", "positif"]
))


# 8. Test manuel


In [ ]:

def predict_mood(text):
    encoding = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    outputs = model(**encoding)
    prediction = torch.argmax(outputs.logits, dim=1).item()

    reverse_map = {0: "negatif", 1: "neutre", 2: "positif"}
    return reverse_map[prediction]

print("\nTest manuel :")
print(predict_mood("Je suis fatigué mais ça va"))
print(predict_mood("Cette journée est incroyable"))
print(predict_mood("Le train arrive à 8h"))
